In [3]:
#いろいろインポート
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ
place='tokuyama'

In [4]:
#モデルスコアデータの読み込み
score_df = pd.read_csv('../csv/model_score_date_{}.csv'.format(place))
score_df=score_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
display(score_df)

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,371150.0,420700.0,406.0,4207.0,88.222011,9.650582,406.0
1,1.0,4.0,52.0,371150.0,420700.0,406.0,4207.0,88.222011,9.650582,406.0
2,1.0,4.0,53.0,371150.0,420700.0,406.0,4207.0,88.222011,9.650582,406.0
3,1.0,4.0,54.0,371150.0,420700.0,406.0,4207.0,88.222011,9.650582,406.0
4,1.0,4.0,55.0,371150.0,420700.0,406.0,4207.0,88.222011,9.650582,406.0
...,...,...,...,...,...,...,...,...,...,...
13855,28.0,8.0,145.0,43480.0,21900.0,17.0,219.0,198.538813,2.739726,6.0
13856,28.0,8.0,146.0,43480.0,19900.0,17.0,199.0,218.492462,3.015075,6.0
13857,28.0,8.0,147.0,43480.0,19900.0,17.0,199.0,218.492462,3.015075,6.0
13858,28.0,8.0,148.0,43480.0,21400.0,17.0,214.0,203.177570,2.803738,6.0


#### モデル性能評価用のデータフレーム作成(result_comごとの閾値を持つ)


In [5]:
#学習データの読み込み
result_df = pd.read_csv('../csv/train_{}_date.csv'.format(place))
result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除

df=result_df
#学習、テストデータ切り分け
num_data = len(df) 
num_valid = int(num_data/6.0)
valid_df = df.loc[(num_data-num_valid):num_data, :]#なるべく最新のデータを未知データに
train_df = df.loc[0:(num_data-num_valid-1), :]
train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])
#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
display(result_com_df)

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1,310,9.696591,939.806452,730.0
1,2,192,6.005630,1044.270833,785.0
2,3,146,4.566781,1502.260274,1095.0
3,4,89,2.783860,2438.314607,1450.0
4,5,229,7.162965,1185.371179,1010.0
...,...,...,...,...,...
110,114,5,0.156397,31912.000000,18970.0
111,115,1,0.031279,9980.000000,9980.0
112,117,2,0.062559,13665.000000,13665.0
113,118,1,0.031279,11810.000000,11810.0


### モデル性能評価機能

In [6]:
#基準をクリアしたモデルを格納するdf
depths_arr=[4,5,6,7,8]
model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for target_com in result_com_df['result_com'].values:
    target_com_df=score_df[score_df['target_com']==target_com]
    #閾値の作成===============================================================
    gain_th=110#利益率の閾値
    result_s=result_com_df[result_com_df['result_com']==target_com]
    buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
    num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
    
    diff_gain_th=108#利益率の閾値
    diff_buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#前後モデルの買ったうちの的中率の閾値
    diff_num_tp_th=result_s['result_com_num'].values[0]*0.13#前後モデルのあたった回数の閾値(出現回数の20%が的中)
    
    #==========================================================
    for target_depth in depths_arr:
        target_com_depth_df=target_com_df[target_com_df['depth']==target_depth]
        
        len_df=len(target_com_depth_df)
        df=target_com_depth_df.copy()
        df['number_i']=np.arange(0,len_df,1)
        
        for _, row in df.iterrows():
            if ((row['number_i']==0) or (row['number_i']==1) or (row['number_i']==len_df-2) or (row['number_i']==len_df-1)):
                pass
            else:
                #前後のモデルのスコア===================================
                diff_m2_row=df[df['number_i']==(row['number_i']-2)]
                diff_m1_row=df[df['number_i']==(row['number_i']-1)]
                diff_p1_row=df[df['number_i']==(row['number_i']+1)]
                diff_p2_row=df[df['number_i']==(row['number_i']+2)]
                diff_models=[diff_m2_row,diff_m1_row,diff_p1_row,diff_p2_row]
                #===================================================
                if ((row['利益率']>gain_th) and (row['購買的中率']>buy_accuracy_th) and (row['的中数']>=num_tp_th)):#初めに真ん中のモデルの性能評価
                    #前後モデルの評価
                    flag_arr=[0,0,0,0]
                    for index in range(len(flag_arr)):

                        diff_model=diff_models[index].iloc[0,:]
                        if ((diff_model['利益率']>diff_gain_th) and (diff_model['購買的中率']>diff_buy_accuracy_th) and (diff_model['的中数']>=diff_num_tp_th)):#前後モデルの性能評価
                            flag_arr[index]=1
                        else:
                            pass
                    if sum(flag_arr)==4:
                        model_para_df=model_para_df.append(row)
                        
                else:
                    pass
        

In [7]:
model_para_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
57,1.0,4.0,108.0,44240.0,39100.0,406.0,391.0,113.145780,17.135550,67.0,57.0
156,1.0,5.0,108.0,59620.0,53900.0,406.0,539.0,110.612245,16.512059,89.0,57.0
157,1.0,5.0,109.0,55940.0,48500.0,406.0,485.0,115.340206,17.319588,84.0,58.0
158,1.0,5.0,110.0,48870.0,39700.0,406.0,397.0,123.098237,18.136020,72.0,59.0
257,1.0,6.0,110.0,58260.0,47800.0,406.0,478.0,121.882845,18.410042,88.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...
13853,28.0,8.0,143.0,43480.0,22800.0,17.0,228.0,190.701754,2.631579,6.0,92.0
13854,28.0,8.0,144.0,43480.0,22800.0,17.0,228.0,190.701754,2.631579,6.0,93.0
13855,28.0,8.0,145.0,43480.0,21900.0,17.0,219.0,198.538813,2.739726,6.0,94.0
13856,28.0,8.0,146.0,43480.0,19900.0,17.0,199.0,218.492462,3.015075,6.0,95.0


In [8]:
model_para_df.to_csv('../csv/{}_good_score_model_date.csv'.format(place), encoding='utf_8_sig')

In [ ]:
diff_model

In [17]:
len_df

99

In [34]:
type(diff_model)

tuple